In [1]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter
%matplotlib qt

In [2]:
metadata = dict(title='Kepler Orbit', artist='Matplotlib')
writer = FFMpegWriter(fps=15, metadata=metadata,bitrate=200000)

In [3]:
mS = 1.99*(10**30) #Sun [kg]
mE = 5.97*(10**24) #Earth [kg]
mK = 1052 #Kepler [kg]
G = 6.67*(10**-11) #gravitational cst [(m^3)(kg^-1)(s^-2)]
R = 147.48*(10**9) #Sun-Earth distance, mag(rS-rE)=R, rE=rS-R [m]

xS = 0 #[m]

xE = 147.48*(10**9) #[m]

xK = 157062804448 #[m]

rCM = np.array([((xE*mE)+(xS*mS))/(mE+mS),0,0])
rS = np.array([xS,0,0]-rCM)
rE = np.array([xE*((2)**0.5)/2,xE*((2)**0.5)/2,0]-rCM)
rK = np.array([xK*0.99,0,xK*0.0078]) #x=xK*cos(0.4474 degrees),z=xK*cos(0.4474 degrees)

print('rCM = '+str(rCM))
print('Shifted Sun Position: rS = ' + str(rS))
print('Shifted Earth Position: rE = ' + str(rE))
print('Kepler Position: rK = ' + str(rK))

rCM = [442438.67268398      0.              0.        ]
Shifted Sun Position: rS = [-442438.67268398       0.               0.        ]
Shifted Earth Position: rE = [1.04283666e+11 1.04284108e+11 0.00000000e+00]
Kepler Position: rK = [1.55492176e+11 0.00000000e+00 1.22508987e+09]


In [4]:
fig_orig = plt.figure()
ax_orig = plt.axes(projection='3d')
# ax_orig.scatter3D(0,0,0)
ax_orig.scatter3D(rCM[0],rCM[1],rCM[2],label='COM')
ax_orig.scatter3D(rS[0],rS[1],rS[2],label='Sun')
ax_orig.scatter3D(rE[0],rE[1],rE[2],label='Earth')
ax_orig.scatter3D(rK[0],rK[1],rK[2],label='Kepler')
plt.legend()
plt.show()

In [5]:
mu = mS*mE/(mS+mE)
w = ((G*mS*mE)/((R**3)*mu))**0.5 #Earth's angular velocity

#v=wr

vS = np.array([0,w*np.linalg.norm(rS),0])
vE = np.array([0,w*np.linalg.norm(rE),0])
vK = np.array([0,w*np.linalg.norm(rK),0])

print('vS = ' + str(vS))
print('vE = ' + str(vE))
print('vK = ' + str(vK))

vS = [0.        0.0900002 0.       ]
vE = [    0.         30000.09436991     0.        ]
vK = [    0.         31630.96449163     0.        ]


In [6]:
y = np.concatenate((rS,rE,rK,vS,vE,vK))
print('y = ' + str(y))

y = [-4.42438673e+05  0.00000000e+00  0.00000000e+00  1.04283666e+11
  1.04284108e+11  0.00000000e+00  1.55492176e+11  0.00000000e+00
  1.22508987e+09  0.00000000e+00  9.00002040e-02  0.00000000e+00
  0.00000000e+00  3.00000944e+04  0.00000000e+00  0.00000000e+00
  3.16309645e+04  0.00000000e+00]


In [7]:
def KeplerODE(t,y):
    global mS,mE,mK,G
    
    rS = y[0:3]
    rE = y[3:6]
    rK = y[6:9]
    vS = y[9:12]
    vE = y[12:15]
    vK = y[15:18]
    
    drSdt = vS
    drEdt = vE
    drKdt = vK
    
    FS = -mS*mE*G*(rS-rE)/np.linalg.norm(rS-rE)**3 + -mS*mK*G*(rS-rK)/np.linalg.norm(rS-rK)**3
    FE = -mE*mS*G*(rE-rS)/np.linalg.norm(rE-rS)**3 + -mE*mK*G*(rE-rK)/np.linalg.norm(rE-rK)**3
    FK = -mK*mS*G*(rK-rS)/np.linalg.norm(rK-rS)**3 + -mK*mE*G*(rK-rE)/np.linalg.norm(rK-rE)**3
    aS = FS/mS
    aE = FE/mE
    aK = FK/mK
    dvSdt = aS
    dvEdt = aE
    dvKdt = aK
    
    return np.concatenate((drSdt,drEdt,drKdt,dvSdt,dvEdt,dvKdt))

In [8]:
semiE = 149.6*(10**9) #semimajor axis Earth
semiK = 1.51*(10**11) #semimajor axis Kepler
PE = ((4*(np.pi**2)*(aE**3))/(G*(mE+mS)))**0.5 #period of Earth [s]
PK = ((4*(np.pi**2)*(aK**3))/(G*(mK+mS)))**0.5 #period of Kepler [s]
print('Period of Earth: ' + str(PE/86400) + ' days')
print('Period of Kepler: ' + str(PK/86400) + ' days')
dt = PK/1000
t = 0
tMax = PK*1.5

xSt = []
ySt = []
zSt = []
xEt = []
yEt = []
zEt = []
xKt = []
yKt = []
zKt = []
tt = []

y = np.concatenate((rS,rE,rK,vS,vE,vK))

fig = plt.figure(dpi=200)
with writer.saving(fig, "fpEarth.mp4", dpi=200):
    while (t<tMax):
        rS = y[0:3]
        rE = y[3:6]
        rK = y[6:9]

        xSt.append(rS[0])
        ySt.append(rS[1])
        zSt.append(rS[2])
        xEt.append(rE[0])
        yEt.append(rE[1])
        zEt.append(rE[2])
        xKt.append(rK[0])
        yKt.append(rK[1])
        zKt.append(rK[2])
        tt.append(t)  

        f1 = KeplerODE(t,y)
        f2 = KeplerODE(t+dt/2.0,y+f1*dt/2.0)
        f3 = KeplerODE(t+dt/2.0,y+f2*dt/2.0)
        f4 = KeplerODE(t+dt,y+f3*dt)

        ax = plt.axes(projection='3d')
        ax.scatter3D(xSt,ySt,zSt,label='Sun' if t == 0 else "")
        ax.scatter3D(xEt,yEt,zEt,label='Earth' if t == 0 else "")
        ax.scatter3D(xKt,yKt,zKt,label='Kepler' if t == 0 else "")
        plt.xlim(-2*(10**11),2*(10**11))
        plt.ylim(-3*(10**11),2*(10**11))
        ax.set_zlim(-1*(10**9),1*(10**9))
        ax.legend()
        
        ax.text(rE[0],rE[1],rE[2],s = 't: ' + str(np.round(t/86400,3)) + ' days')
        y = y + (f1 + 2.0*f2 + 2.0*f3 + f4) / 6.0 * dt
        t = t + dt
        writer.grab_frame()

Period of Earth: 365.2358585036019 days
Period of Kepler: 370.375364297352 days


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles wi

4

1.0

95.0